# CNN Model Template

This notebook is a template for building a CNN model.

## Imports and inital setup

In [22]:
# Imports
import pandas as pd

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras import layers
from tensorflow.keras.models import Model

import sys

sys.path.append("..")
from helperfunctions import modelhelper as mh
from helperfunctions import imagehelper as ih

SEED = 856
NUM_EPOCHS = 10

# File path variables
# please make sure to use the correct path to the meta data file

FILEPATH_JPGS = './../data/jpgs/'
FILEPATH_PROCESSED="./../data/processed/"
FILEPATH_OUTPUT = './../data/jpgs/'  # Replace with your folder path

TARGET_LABEL="dx_binary"

IMAGE_SIZE = (224, 224)

### Loading (augmented) metadata as test, train, validation from files

In [23]:
# Read the metadata file
train_df = pd.read_csv(FILEPATH_PROCESSED+"train_from_Metadata_processed.csv")
validation_df = pd.read_csv(FILEPATH_PROCESSED+"validation_from_Metadata_processed.csv")
test_df = pd.read_csv(FILEPATH_PROCESSED+"test_from_Metadata_processed.csv")

train_df.sample(15)

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,dx_binary,image_path
3165,HAM_0004141,ISIC_0030705.jpg,bkl,histo,50.0,female,back,vidir_molemax,not_skin_cancer,./../data/jpgs/ISIC_0030705.jpg
2217,HAM_0001326,aug_5Kr5rvISIC_0031824.jpg,bcc,histo,60.0,male,scalp,vidir_modern,skin_cancer,./../data/jpgs/aug_5Kr5rvISIC_0031824.jpg
4554,HAM_0004960,ISIC_0029967.jpg,df,histo,65.0,female,lower extremity,rosendahl,not_skin_cancer,./../data/jpgs/ISIC_0029967.jpg
1445,HAM_0003212,aug_LtxRY8ISIC_0028519.jpg,mel,histo,80.0,male,back,vidir_modern,skin_cancer,./../data/jpgs/aug_LtxRY8ISIC_0028519.jpg
1680,HAM_0000496,ISIC_0029545.jpg,bcc,histo,55.0,female,chest,rosendahl,skin_cancer,./../data/jpgs/ISIC_0029545.jpg
291,HAM_0003796,ISIC_0030696.jpg,nv,follow_up,50.0,male,trunk,vidir_molemax,not_skin_cancer,./../data/jpgs/ISIC_0030696.jpg
4795,HAM_0002129,aug_BhVv8ZISIC_0025903.jpg,df,consensus,60.0,male,abdomen,vidir_molemax,not_skin_cancer,./../data/jpgs/aug_BhVv8ZISIC_0025903.jpg
1513,HAM_0007313,ISIC_0025299.jpg,bcc,histo,70.0,male,back,rosendahl,skin_cancer,./../data/jpgs/ISIC_0025299.jpg
4118,HAM_0004943,aug_OfHgL8ISIC_0027790.jpg,vasc,consensus,50.0,female,face,vidir_molemax,not_skin_cancer,./../data/jpgs/aug_OfHgL8ISIC_0027790.jpg
5047,HAM_0005918,aug_etPikCISIC_0029177.jpg,df,consensus,35.0,female,lower extremity,vidir_molemax,not_skin_cancer,./../data/jpgs/aug_etPikCISIC_0029177.jpg


## Setting up the image data generator for training and validation

In [24]:
def custom_preprocessing(np_image, image_size):
    np_image = ih.center_crop_image(np_image) # Crop image to square format
    np_image = ih.resize_as_preprocess(np_image, image_size) # resize the image
    np_image = preprocess_input(np_image) # Apply the preprocess_input function of MobileNetV3Large
    return np_image

In [25]:
# Setting up the Image Data Generator for the train data set

datagen_train = ImageDataGenerator(
    rescale=1.0 / 255.0, # Rescale pixel values to [0, 1], important for CNNs to perform better
    preprocessing_function=lambda x: custom_preprocessing(x, IMAGE_SIZE), # Apply the custom preprocessing function 
)

datagen_validation = ImageDataGenerator(
    rescale=1.0 / 255.0,
    preprocessing_function=lambda x: custom_preprocessing(x, IMAGE_SIZE)
)

validation_generator = datagen_validation.flow_from_dataframe(
    dataframe=validation_df,
    directory=FILEPATH_JPGS,
    x_col="image_id",
    y_col=TARGET_LABEL,
    class_mode="categorical",
    target_size=IMAGE_SIZE,
    batch_size=32
)

train_data_generator = datagen_train.flow_from_dataframe(
        dataframe=train_df,
        directory=FILEPATH_JPGS,
        x_col="image_id",
        y_col=TARGET_LABEL,
        class_mode="categorical",
        target_size=IMAGE_SIZE,
        batch_size=32
)

Found 2003 validated image filenames belonging to 2 classes.
Found 5256 validated image filenames belonging to 2 classes.


# Using MobileNet V3 large

In [26]:
PRETRAINED = True

if PRETRAINED:
    # Initialize the MobileNetV3Large model
    base_model = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

    # Freeze the base model's layers to use it for feature extraction
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom layers on top 
    x = layers.Conv2D(128, (3, 3), activation='relu')(base_model.output)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(train_df[TARGET_LABEL].nunique(), activation='softmax')(x) 

    # Create the full model
    model = Model(inputs=base_model.input, outputs=x, name="MobilneNetV3Large_pretrained-weights_binary_fixed-layers_custom-conv2D")

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.legacy.Adam(),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

    model.summary()

else:
    # Initialize the MobileNetV3Large model
    base_model = MobileNetV3Large(weights=None, include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

    # Freeze the base model's layers to use it for feature extraction
    for layer in base_model.layers:
        layer.trainable = True

    # Add custom layers on top 
    x = layers.Flatten()(base_model.output)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(train_df[TARGET_LABEL].nunique(), activation='softmax')(x) 

    # Create the full model
    model = Model(inputs=base_model.input, outputs=x, name="MobilneNetV3Large_rand-weights")

    model.compile(optimizer=tf.keras.optimizers.legacy.Adam(),
            loss='categorical_crossentropy',
            metrics=['accuracy'])

    model.summary()


Model: "MobilneNetV3Large_pretrained-weights_binary_fixed-layers_custom-conv2D"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling_4 (Rescaling)     (None, 224, 224, 3)          0         ['input_5[0][0]']             
                                                                                                  
 Conv (Conv2D)               (None, 112, 112, 16)         432       ['rescaling_4[0][0]']         
                                                                                                  
 Conv/BatchNorm (BatchNorma  (None, 112, 112, 16)         64        ['Conv[0][0]']                
 lization)                   

## Model Training

In [27]:
# Train the model

history = model.fit(
    train_data_generator,
    epochs=NUM_EPOCHS,              
    verbose=1,                      # Adjust verbosity level
    batch_size=None,                # Set the batch size, default is 32, can be increased to speed up training
    callbacks=None,                 # List of callbacks to apply during training 
    validation_split=0.0,           # not needed as we use a validation data generator
    validation_data=validation_generator,
    shuffle=True,                   # Shuffle the training data before each epoch
    sample_weight=None,             # Set the weights for the train data set !
    class_weight=None,              # Set the weights for the classes, not needed if we use sample weights
    initial_epoch=0,                # Use this to continue training from a specific epoch
    steps_per_epoch=None,           # Set the number of steps per epoch, default is len(x_train) // batch_size
    validation_steps=None,          # Set the number of steps for validation, default is len(x_val) // batch_size
    validation_batch_size=None,     # Set the batch size for validation, default is batch_size
    validation_freq=1,              # Only relevant if validation data is a generator. Set the frequency to validate the model on the validation set
    max_queue_size=10,              # Set the max size for the generator queue
    workers=-1,                     # Set the max number of processes to generate the data in parallel, -1 means all CPUs
    use_multiprocessing=False       # Set to True if you use a generator in parallel, e.g. model.predict_generator()
)

Epoch 1/10
165/165 [==============================] - 80s 461ms/step - loss: 0.7236 - accuracy: 0.5580 - val_loss: 0.6016 - val_accuracy: 0.8048
Epoch 2/10
165/165 [==============================] - 87s 527ms/step - loss: 0.6846 - accuracy: 0.5713 - val_loss: 0.6268 - val_accuracy: 0.8048
Epoch 3/10
165/165 [==============================] - 117s 709ms/step - loss: 0.6840 - accuracy: 0.5713 - val_loss: 0.6252 - val_accuracy: 0.8048
Epoch 4/10
165/165 [==============================] - 102s 616ms/step - loss: 0.6837 - accuracy: 0.5713 - val_loss: 0.6231 - val_accuracy: 0.8048
Epoch 5/10
165/165 [==============================] - 115s 700ms/step - loss: 0.6836 - accuracy: 0.5713 - val_loss: 0.6241 - val_accuracy: 0.8048
Epoch 6/10
165/165 [==============================] - 95s 578ms/step - loss: 0.6833 - accuracy: 0.5713 - val_loss: 0.6149 - val_accuracy: 0.8048
Epoch 7/10
165/165 [==============================] - ETA: 0s - loss: 0.6833 - accuracy: 0.5713

KeyboardInterrupt: 

## Plotting accuracy and loss of train and validation set

In [ ]:
mh.model_plot_accuracy(history)

## Evaluating the model on the test set

In [ ]:
mh.model_accuracy_on_test(model, test_df, TARGET_LABEL, IMAGE_SIZE)

## Saving the model

In [ ]:
from datetime import datetime
timestamp = datetime.now()
model_path = f"../models/model_{timestamp}.h5"
model.save(model_path)